import libraries

In [1]:
import os
import pyodbc
import pandas as pd
import warnings
from datetime import datetime

warnings.simplefilter(action="ignore", category=UserWarning)

Variables

In [2]:
conn = None
raw_file_path = 'C:/DB/raw_csv_files'

if not os.path.exists(raw_file_path):
    os.makedirs(raw_file_path)

import properties

In [3]:
import configparser

config = configparser.ConfigParser()
config.read("config.ini")

# Access values
db_dsn = config.get("DATABASE", "dsn")
db_user = config.get("DATABASE", "username")
db_pass = config.get("DATABASE", "password")
last_run_date = config.get("SETTINGS", "last_run_date")

Create an odbc connection

In [4]:
try:
    conn = pyodbc.connect(f'DSN={db_dsn};UID={db_user};PWD={db_pass}')

except pyodbc.Error as e:
    print("Sage50Accounts Database connection error:", e)

Read each tables and covert it to csv

In [5]:
def odbc_to_csv():         
        cursor = conn.cursor()
        # tables = cursor.tables()
        tables = [row.table_name for row in cursor.tables(tableType="TABLE")]
        print("\nSage50Accounts Tables:")
        for table in tables:
            table_name = table
            output_csv_path = f"{raw_file_path}/{table_name}.csv"            

            try:

                cursor.execute(f"SELECT * FROM {table_name} WHERE 1=0") 
                all_columns = [column[0] for column in cursor.description]

                exclude_columns = []

                selected_columns = [col for col in all_columns if col not in exclude_columns]

                df = pd.read_sql(f'SELECT {', '.join(selected_columns)} FROM {table_name} WHERE RECORD_CREATE_DATE > "{last_run_date}"', conn)

            except Exception as e:
                print(f"Error in {table_name} table export:", e)
                

            df.to_csv(output_csv_path, index=False)
            print(f"Table '{table_name}' exported to '{output_csv_path}' successfully.") 


        current_date = datetime.today().strftime("%Y-%m-%d %H:%M:%S")
        config.set("SETTINGS", "last_run_date", current_date)

        # Write changes back to the file
        with open("config.ini", "w") as configfile:
            config.write(configfile)

        print("Updated 'last_run_date' in SETTINGS section.") 
        

In [6]:
if conn:
    odbc_to_csv()



Sage50Accounts Tables:
Table 'COMPANY' exported to 'C:/DB/raw_csv_files/COMPANY.csv' successfully.
Table 'PERIOD' exported to 'C:/DB/raw_csv_files/PERIOD.csv' successfully.
Table 'SALES_LEDGER' exported to 'C:/DB/raw_csv_files/SALES_LEDGER.csv' successfully.
Table 'PURCHASE_LEDGER' exported to 'C:/DB/raw_csv_files/PURCHASE_LEDGER.csv' successfully.
Table 'NOMINAL_LEDGER' exported to 'C:/DB/raw_csv_files/NOMINAL_LEDGER.csv' successfully.
Table 'AUDIT_HEADER' exported to 'C:/DB/raw_csv_files/AUDIT_HEADER.csv' successfully.
Table 'AUDIT_SPLIT' exported to 'C:/DB/raw_csv_files/AUDIT_SPLIT.csv' successfully.
Table 'AUDIT_HISTORY_HEADER' exported to 'C:/DB/raw_csv_files/AUDIT_HISTORY_HEADER.csv' successfully.
Table 'AUDIT_HISTORY_SPLIT' exported to 'C:/DB/raw_csv_files/AUDIT_HISTORY_SPLIT.csv' successfully.
Table 'AUDIT_HISTORY_JOURNAL' exported to 'C:/DB/raw_csv_files/AUDIT_HISTORY_JOURNAL.csv' successfully.
Table 'AUDIT_HISTORY_USAGE' exported to 'C:/DB/raw_csv_files/AUDIT_HISTORY_USAGE.c

Close connection

In [7]:
if conn:
    conn.close()
    print("Connection closed.")

Connection closed.
